In [22]:
import numpy as np
from nptdms import TdmsFile
import os
from pathlib import Path

path_base_folder = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data"
output_base_folder = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data"

patient_path_1 = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 1\recording 2\Patient 1_2.tdms"

Patients_to_use = ["Patient 3", "Patient 5", "Patient 6", "Patient 8", "Patient 10", "Patient 14", "Patient 15", "Patient 16", "Patient 21", "Patient 23", "Patient 27", "Patient 28", "Patient 29", "Patient 31", "Patient 34", "Patient 37", "Patient 39", "Patient 40", "Patient 41", "Patient 42"]

In [23]:
def inspect_tdms(tdms_path):
    tdms_path = Path(tdms_path)
    with TdmsFile.read(tdms_path) as tdms:
        print(f"\n=== FILE: {tdms_path.name} ===")
        print("File properties:")
        for k, v in tdms.properties.items():
            print(f"  - {k}: {v}")
        for group in tdms.groups():
            print(f"\n[Group] {group.name}")
            # (Gruppe-properties kan ikke altid skrives 1:1 ud igen; vi bibeholder primært fil- og kanal-properties)
            for ch in group.channels():
                print(f"  [Channel] {ch.name}")
                # Vis udvalgte properties
                prop_keys = list(ch.properties.keys())
                interesting = [p for p in prop_keys if p.lower() in {"wf_start_time", "wf_increment", "unit_string", "name"}]
                for p in interesting:
                    print(f"    - {p}: {ch.properties[p]}")
                # Vis antal samples
                try:
                    n = ch._length  # hurtigt, uden at læse alt
                except Exception:
                    n = len(ch[:])   # fallback
                print(f"    - samples: {n}")

In [24]:
final_patient_folders = []
folders = [folder for folder in os.listdir(path_base_folder) if folder in Patients_to_use]

for folder in folders:
    folder_path = os.path.join(path_base_folder, folder)
    output_path = os.path.join(output_base_folder, folder)
    os.makedirs(output_path, exist_ok=True)

    for root, dirs, files in os.walk(folder_path):
        if not dirs:  # Deepest folder
            final_patient_folders.append(root)

print("Final patient folders:", final_patient_folders)

Final patient folders: ['D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 10\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 14\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 14\\recording 2', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 15\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 15\\recording 2', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 16\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 21\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatc

In [25]:
for folder1 in final_patient_folders[0:1]:  # Test med én mappe først
    files_in_folder = os.listdir(folder1)
    tdms_files = [f for f in files_in_folder if f.endswith('.tdms')]
    if tdms_files:
        path_tdms = os.path.join(folder1, tdms_files[0])
        inspect_tdms(path_tdms)
        tdms_file = TdmsFile.read(path_tdms)

        print(folder1)
        parent_folder = os.path.basename(os.path.dirname(folder1))
        print(parent_folder)
        parent_subfolder = os.path.basename(folder1)
        output_path_file = os.path.join(output_base_folder, parent_folder, parent_subfolder, tdms_files[0])
        print("Output path for corrected file:", output_path_file)

        # tdms_file_path = os.path.join(folder1, tdms_files[0])
        # output = os.path.join(output_base_folder, os.path.basename(folder1), tdms_files[0])
        # x = correct_tdms_start_time(tdms_file_path, output)
        # print("TDMS file path:", tdms_file_path)
        # print("Output path:", output)
        # print("Corrected file:", x)
        # print("TDMS files:", tdms_files)



=== FILE: Patient 10_1.tdms ===
File properties:
  - name: Sidsel Borg Nielsen 20062017 kl_134443 __23062017_kl_130000.tdms
  - author: Jesper
  - description: 
  - registertxt1: Written by National Instruments LabVIEW

[Group] Untitled
  [Channel] EKG
    - wf_start_time: 2017-06-20T11:44:43.000000
    - wf_increment: 0.001953125
    - unit_string: uV
    - samples: 132710144
D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 10\recording 1
Patient 10
Output path for corrected file: D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms


In [26]:
tdms_file.properties["name"] = "Patient 10_1 20_06_2017 kl_13_44_43 __23_06_2017 kl_13_00_00"
tdms_file.properties["Corrected_by: "] = "Kasper Viken Jensen"
print("Updated file properties:")
for key, value in tdms_file.properties.items():
    print(f"  {key}: {value}")

Updated file properties:
  name: Patient 10_1 20_06_2017 kl_13_44_43 __23_06_2017 kl_13_00_00
  author: Jesper
  description: 
  registertxt1: Written by National Instruments LabVIEW
  Corrected_by: : Kasper Viken Jensen


In [27]:
timestamp_corrected1 = np.datetime64('2017-06-20T13:44:43.000000')

channel1 = tdms_file.groups()[0].channels()[0]
print("Channel properties before correction:")
for key, value in channel1.properties.items():
    print(f"  {key}: {value}")

Channel properties before correction:
  wf_start_time: 2017-06-20T11:44:43.000000
  wf_start_offset: 0.0
  wf_increment: 0.001953125
  wf_samples: 132710144
  DigitalMaximum: 32767
  DigitalMinimum: -32768
  NI_ChannelName: EKG
  NI_ExpIsRelativeTime: False
  wf_time_pref: absolute
  NI_ExpStartTimeStamp: 2017-06-20T11:44:43.000000
  NI_ExpTimeStamp: 2017-06-20T11:44:43.000000
  NI_ExpXDimension: t
  wf_xname: Time
  wf_xunit_string: s
  NI_UnitDescription: uV
  unit_string: uV
  PhysicalMaximum: 90298.0
  PhysicalMinimum: -90298.0
  PreFilter: 
  TransducerType: 
  ValsPerRec: 128


In [28]:

tdms_file.groups()[0].channels()[0].properties["wf_start_time"] = timestamp_corrected1
tdms_file.groups()[0].channels()[0].properties["NI_ExpStartTimeStamp"] = timestamp_corrected1
tdms_file.groups()[0].channels()[0].properties["NI_ExpTimeStamp"] = timestamp_corrected1

print("Channel properties after correction:")
for key, value in channel1.properties.items():
    print(f"  {key}: {value}")

Channel properties after correction:
  wf_start_time: 2017-06-20T13:44:43.000000
  wf_start_offset: 0.0
  wf_increment: 0.001953125
  wf_samples: 132710144
  DigitalMaximum: 32767
  DigitalMinimum: -32768
  NI_ChannelName: EKG
  NI_ExpIsRelativeTime: False
  wf_time_pref: absolute
  NI_ExpStartTimeStamp: 2017-06-20T13:44:43.000000
  NI_ExpTimeStamp: 2017-06-20T13:44:43.000000
  NI_ExpXDimension: t
  wf_xname: Time
  wf_xunit_string: s
  NI_UnitDescription: uV
  unit_string: uV
  PhysicalMaximum: 90298.0
  PhysicalMinimum: -90298.0
  PreFilter: 
  TransducerType: 
  ValsPerRec: 128


In [ ]:
# for group in tdms_file.groups():
#     print(f"Group: {group.name}")
#     for channel in group.channels():
#         print(f"  Channel: {channel.name}")
#         for key, value in channel.properties.items():
#             print(f"    {key}: {value}")

Group: Untitled
  Channel: EKG
    wf_start_time: 2017-06-20T13:44:43.000000
    wf_start_offset: 0.0
    wf_increment: 0.001953125
    wf_samples: 132710144
    DigitalMaximum: 32767
    DigitalMinimum: -32768
    NI_ChannelName: EKG
    NI_ExpIsRelativeTime: False
    wf_time_pref: absolute
    NI_ExpStartTimeStamp: 2017-06-20T13:44:43.000000
    NI_ExpTimeStamp: 2017-06-20T13:44:43.000000
    NI_ExpXDimension: t
    wf_xname: Time
    wf_xunit_string: s
    NI_UnitDescription: uV
    unit_string: uV
    PhysicalMaximum: 90298.0
    PhysicalMinimum: -90298.0
    PreFilter: 
    TransducerType: 
    ValsPerRec: 128


In [ ]:
# from nptdms import TdmsWriter, RootObject
# import os

# print("Writing corrected TDMS file to:", output_path_file)

# # Ensure the parent directory exists
# os.makedirs(os.path.dirname(output_path_file), exist_ok=True)

# from nptdms import RootObject
# import csv

# original_groups = tdms_file.groups()
# original_channels = [chan for chan in original_groups[0].channels()]

# # for chan in original_channels:
# #     print(f"Channel: {chan.name}")
# #     for key, value in chan.properties.items():
# #         print(f"  {key}: {value}")

# # with TdmsWriter(output_path_file) as writer:
# #     # Create RootObject with updated file properties
# #     root_obj = RootObject(tdms_file.properties)
# #     # Get all groups and channels from the original file
# channels_to_copy = [chan for chan in original_channels]
# #     writer.write_segment([root_obj] + original_groups + channels_to_copy)

# # for chan in channels_to_copy:
# #     print(f"Channel: {chan.name}")
# #     for key, value in chan.properties.items():
# #         print(f"  {key}: {value}")

# # Write raw data to CSV
# rawdata_csv_path = output_path_file.replace('.tdms', '_rawdata.csv')
# channel = channels_to_copy[0]
# np.savetxt(rawdata_csv_path, channel[:], delimiter=",")
# print(f"Raw data written to: {rawdata_csv_path}")

# # Write group, channel data and properties to another CSV
# meta_csv_path = output_path_file.replace('.tdms', '_meta.csv')
# with open(meta_csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['Group', 'Channel', 'Property', 'Value'])
#     for group in tdms_file.groups():
#         for chan in group.channels():
#             for key, value in chan.properties.items():
#                 writer.writerow([group.name, chan.name, key, value])
# print(f"Group/channel properties written to: {meta_csv_path}")


Writing corrected TDMS file to: D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms


KeyboardInterrupt: 

In [ ]:
# channels_to_copy = [chan for chan in tdms_file.groups()[0].channels()]
# print("Channels to copy:", [chan.name for chan in channels_to_copy])
# for chan in channels_to_copy:
#     print(f"Channel: {chan.name}")
#     for key, value in chan.properties.items():
#         print(f"  {key}: {value}")

Channels to copy: ['EKG']
Channel: EKG
  wf_start_time: 2017-06-20T13:44:43.000000
  wf_start_offset: 0.0
  wf_increment: 0.001953125
  wf_samples: 132710144
  DigitalMaximum: 32767
  DigitalMinimum: -32768
  NI_ChannelName: EKG
  NI_ExpIsRelativeTime: False
  wf_time_pref: absolute
  NI_ExpStartTimeStamp: 2017-06-20T13:44:43.000000
  NI_ExpTimeStamp: 2017-06-20T13:44:43.000000
  NI_ExpXDimension: t
  wf_xname: Time
  wf_xunit_string: s
  NI_UnitDescription: uV
  unit_string: uV
  PhysicalMaximum: 90298.0
  PhysicalMinimum: -90298.0
  PreFilter: 
  TransducerType: 
  ValsPerRec: 128


In [ ]:
# # Inspect the corrected file
# # inspect_tdms(output_path_file)

# tdms_corrected_1 = TdmsFile(output_path_file)

# print("Inspecting corrected TDMS file:")
# for group in tdms_corrected_1.groups()[0:1]:
#     print(f"Group: {group.name}")
#     for channel in group.channels():
#         print(f"  Channel: {channel.name}")
#         for key, value in channel.properties.items():
#             print(f"    {key}: {value}")

Inspecting corrected TDMS file:
Group: Untitled


In [ ]:
# from nptdms import TdmsFile, TdmsWriter, RootObject

# original_file = TdmsFile(r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms")
# original_groups = original_file.groups()
# original_channels = [chan for group in original_groups for chan in group.channels()]

# with TdmsWriter(r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms") as copied_file:
#     root_object = RootObject(original_file.properties)
#     channels_to_copy = [chan for chan in original_channels]
#     copied_file.write_segment([root_object] + original_groups + channels_to_copy)

In [ ]:
# corrected_file21 = TdmsFile(r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms")

# groups = corrected_file21.groups()
# channels = [chan for group in groups for chan in group.channels()]

# for group in groups:
#     print(f"Group: {group.name}")
#     for channel in group.channels():
#         print(f"  Channel: {channel.name}")
#         for key, value in channel.properties.items():
#             print(f"    {key}: {value}")

Group: Untitled


In [ ]:
# from nptdms import TdmsFile, TdmsWriter
# from nptdms.writer import RootObject, GroupObject, ChannelObject
# from pathlib import Path
# import numpy as np

# src = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms"
# dst = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms"

# Path(dst).parent.mkdir(parents=True, exist_ok=True)

# with TdmsFile.read(src) as tf:
#     # Fil-properties (du kan tilføje/rette her)
#     file_props = dict(tf.properties) if tf.properties else {}
#     file_props["Corrected_by"] = "Kasper Viken Jensen"

#     writer_objs = [RootObject(file_props)]

#     # Kopiér alle grupper og kanaler
#     for g in tf.groups():
#         # group-properties hvis du vil bevare dem:
#         g_props = dict(getattr(g, "properties", {}) or {})
#         writer_objs.append(GroupObject(g.name, properties=g_props))

#         for ch in g.channels():
#             data = ch[:]                      # <-- HENTER SAMPLES
#             props = dict(ch.properties) or {} # kopiér kanal-properties

#             # Gør tidsfelter skrivbare (numpy.datetime64 anbefales)
#             for k in ("wf_start_time", "NI_ExpStartTimeStamp", "NI_ExpTimeStamp"):
#                 if k in props and isinstance(props[k], str):
#                     try:
#                         props[k] = np.datetime64(props[k])
#                     except Exception:
#                         pass

#             # wf_samples udledes af len(data) – fjern hvis til stede
#             props.pop("wf_samples", None)

#             writer_objs.append(ChannelObject(g.name, ch.name, data, properties=props))

# # Skriv hele segmentet på én gang
# with TdmsWriter(dst) as w:
#     w.write_segment(writer_objs)

# print(f"[OK] Skrev TDMS med {sum(isinstance(o, ChannelObject) for o in writer_objs)} kanal(er) → {dst}")


[OK] Skrev TDMS med 1 kanal(er) → D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms


In [ ]:
# from nptdms import TdmsFile, TdmsWriter
# from nptdms.writer import RootObject, GroupObject, ChannelObject
# from pathlib import Path
# import numpy as np

# def copy_tdms(src_path, dst_path, corrected_by=None):
#     src = Path(src_path); dst = Path(dst_path)
#     dst.parent.mkdir(parents=True, exist_ok=True)

#     with TdmsFile.read(src) as tf, TdmsWriter(dst) as writer:
#         file_props = dict(tf.properties) if tf.properties else {}
#         if corrected_by:
#             file_props["Corrected_by"] = corrected_by

#         writer_objs = [RootObject(file_props)]
#         channel_names = []  # til debug-print

#         for g in tf.groups():
#             g_props = dict(getattr(g, "properties", {}) or {})
#             writer_objs.append(GroupObject(g.name, properties=g_props))

#             for ch in g.channels():
#                 # DETACH data fra memmap
#                 data = np.array(ch[:], copy=True)

#                 # props = dict(ch.properties) or {}
#                 props = {}
#                 # normalisér tidsfelter
#                 for k in ("wf_start_time", "NI_ExpStartTimeStamp", "NI_ExpTimeStamp"):
#                     if k in props and isinstance(props[k], str):
#                         try:
#                             props[k] = np.datetime64(props[k])
#                         except Exception:
#                             pass
#                 props.pop("wf_samples", None)

#                 writer_objs.append(ChannelObject(g.name, ch.name, data, properties=props))
#                 channel_names.append((g.name, ch.name, len(data), data.dtype))

#         print(f"[DEBUG] Planlægger at skrive {len(channel_names)} kanal(er):")
#         for gn, cn, n, dt in channel_names:
#             print(f"  - {gn}/{cn}: samples={n}, dtype={dt}")

#         if not channel_names:
#             raise RuntimeError("Ingen kanaler fundet at skrive. Tjek at kildefilen faktisk indeholder kanaler.")

#         writer.write_segment(writer_objs)

#     print(f"[OK] Skrev {len(channel_names)} kanal(er) til: {dst}")

# # Eksempel-kald:
# copy_tdms(
#     r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms",
#     r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms",
#     corrected_by="Kasper Viken Jensen"
# )


[DEBUG] Planlægger at skrive 1 kanal(er):
  - Untitled/EKG: samples=132710144, dtype=float64
[OK] Skrev 1 kanal(er) til: D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms


In [ ]:
# with TdmsFile.read(r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms") as t:
#     for g in t.groups():
#         print("[Group]", g.name)
#         for c in g.channels():
#             print("  [Channel]", c.name, "samples:", len(c[:]))
#             for k in ("wf_start_time","wf_increment","unit_string","NI_ChannelName"):
#                 if k in c.properties:
#                     print("    -", k, ":", c.properties[k])


[Group] Untitled


In [ ]:
from nptdms import TdmsFile, TdmsWriter
from nptdms.writer import RootObject, GroupObject, ChannelObject
from pathlib import Path
import numpy as np

def tdms_min_copy(src_path, dst_path, corrected_by=None):
    src, dst = Path(src_path), Path(dst_path)
    dst.parent.mkdir(parents=True, exist_ok=True)

    with TdmsFile.read(src) as tf, TdmsWriter(dst) as writer:
        file_props = dict(tf.properties) if tf.properties else {}
        if corrected_by:
            file_props["Corrected_by"] = corrected_by

        objs = [RootObject(file_props)]
        ch_count = 0
        for g in tf.groups():
            objs.append(GroupObject(g.name))
            for ch in g.channels():
                data = np.array(ch[:], copy=True)  # detach fra memmap
                objs.append(ChannelObject(g.name, ch.name, data, properties={}))  # <-- ingen props
                ch_count += 1

        print(f"[DEBUG] minimal write: channels={ch_count}")
        writer.write_segment(objs)

    print(f"[OK] Minimal kopi skrevet → {dst}")


# Kør denne med dine stier
src = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms"
dst_min = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1__mincopy.tdms"
tdms_min_copy(src, dst_min, corrected_by="Kasper Viken Jensen")

# Verificér: bør nu vise kanaler
with TdmsFile.read(dst_min) as t:
    for g in t.groups():
        print("[Group]", g.name)
        for c in g.channels():
            print("  [Channel]", c.name, "samples:", len(c[:]))


[DEBUG] minimal write: channels=1
[OK] Minimal kopi skrevet → D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1__mincopy.tdms
[Group] Untitled
  [Channel] EKG samples: 132710144


In [ ]:
from nptdms import TdmsFile, TdmsWriter
from nptdms.writer import RootObject, GroupObject, ChannelObject
from pathlib import Path
import numpy as np
import tempfile, os

def _safe_cast_prop(k, v):
    """Gør properties 'writer-venlige'."""
    import numpy as np
    # Tillad primitive typer
    if isinstance(v, (str, int, float, bool)):
        return v
    # Datotider → numpy.datetime64 hvis mulig
    for dt_type in ("datetime", "Timestamp"):
        if dt_type in type(v).__name__:
            try: return np.datetime64(str(v))
            except Exception: pass
    # numpy scalars → Python scalars
    if hasattr(v, "item"):
        try: return v.item()
        except Exception: pass
    # arrays/lister → skip (TDMS props er skalarer)
    if isinstance(v, (list, tuple, np.ndarray)):
        raise TypeError("sequence not allowed")
    # fallback → str
    return str(v)

def find_bad_props(src_path, sample_len=2000):
    src = Path(src_path)
    bad = []
    good = []
    with TdmsFile.read(src) as tf:
        g = tf.groups()[0]
        ch = g.channels()[0]
        data_small = np.array(ch[:sample_len], copy=True)

        all_props = dict(ch.properties) or {}
        # Gå alle keys igennem enkeltvist
        for k, v in all_props.items():
            tmpfile = Path(tempfile.gettempdir())/f"tdms_prop_test_{k}.tdms"
            try:
                cast_v = _safe_cast_prop(k, v)
                with TdmsWriter(tmpfile) as w:
                    objs = [
                        RootObject({}),
                        GroupObject(g.name),
                        ChannelObject(g.name, ch.name, data_small, properties={k: cast_v})
                    ]
                    w.write_segment(objs)
                # læs tilbage og tjek at kanalen findes
                with TdmsFile.read(tmpfile) as t2:
                    ok = any(True for _g in t2.groups() for _ in _g.channels())
                if ok:
                    good.append(k)
                else:
                    bad.append(k)
            except Exception as e:
                bad.append((k, repr(e)))
            finally:
                try: os.remove(tmpfile)
                except Exception: pass
    return good, bad

good_props, bad_props = find_bad_props(src)
print("[OK] Good props:", good_props)
print("[!!] Bad props:", bad_props)


[OK] Good props: ['wf_start_time', 'wf_start_offset', 'wf_increment', 'wf_samples', 'DigitalMaximum', 'DigitalMinimum', 'NI_ChannelName', 'NI_ExpIsRelativeTime', 'wf_time_pref', 'NI_ExpStartTimeStamp', 'NI_ExpTimeStamp', 'NI_ExpXDimension', 'wf_xname', 'wf_xunit_string', 'NI_UnitDescription', 'unit_string', 'PhysicalMaximum', 'PhysicalMinimum', 'PreFilter', 'TransducerType', 'ValsPerRec']
[!!] Bad props: []


In [95]:
from nptdms import TdmsFile, TdmsWriter
from nptdms.writer import RootObject, GroupObject, ChannelObject
from pathlib import Path
import numpy as np

def write_final_with_whitelist(src_path, dst_path, good_keys, corrected_by=None, new_start_time=None):
    src, dst = Path(src_path), Path(dst_path)
    dst.parent.mkdir(parents=True, exist_ok=True)

    def safe_cast_all(props: dict):
        out = {}
        for k, v in props.items():
            if k not in good_keys:
                continue
            # cast som i scanneren
            try:
                out[k] = _safe_cast_prop(k, v)
            except Exception:
                continue
        # evt. overskriv starttid
        if new_start_time is not None:
            out["wf_start_time_original"] = props.get("wf_start_time", None)
            out["wf_start_time"] = np.datetime64(new_start_time)
        # fjern 'wf_samples' – udledes af datas længde
        out.pop("wf_samples", None)
        return out

    with TdmsFile.read(src) as tf, TdmsWriter(dst) as writer:
        file_props = dict(tf.properties) if tf.properties else {}
        if corrected_by:
            file_props["Corrected_by"] = corrected_by

        objs = [RootObject(file_props)]
        ch_count = 0

        for g in tf.groups():
            objs.append(GroupObject(g.name))
            for ch in g.channels():
                data = np.array(ch[:], copy=True)
                props = safe_cast_all(dict(ch.properties) or {})
                objs.append(ChannelObject(g.name, ch.name, data, properties=props))
                ch_count += 1

        writer.write_segment(objs)

    print(f"[OK] Skrev {ch_count} kanal(er) til: {dst}")

# Kør den endelige skrivning:
dst_final = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1__final.tdms"
write_final_with_whitelist(
    src_path=src,
    dst_path=dst_final,
    good_keys=set(good_props),             # fra Celle 2
    corrected_by="Kasper Viken Jensen",
    new_start_time=None                    # eller fx "2017-06-20T13:44:43"
)

# Verificér
with TdmsFile.read(dst_final) as t:
    for g in t.groups():
        print("[Group]", g.name)
        for c in g.channels():
            print("  [Channel]", c.name, "samples:", len(c[:]))
            for k in c.properties:
                print(f"    - {k}: {c.properties[k]}")
            # for k in ("wf_start_time","wf_increment","unit_string","NI_ChannelName"):
            #     if k in c.properties:
            #         print("    -", k, ":", c.properties[k])


[OK] Skrev 1 kanal(er) til: D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1__final.tdms
[Group] Untitled
  [Channel] EKG samples: 132710144
    - wf_start_time: 2017-06-20T11:44:43.000000
    - wf_start_offset: 0.0
    - wf_increment: 0.001953125
    - DigitalMaximum: 32767
    - DigitalMinimum: -32768
    - NI_ChannelName: EKG
    - NI_ExpIsRelativeTime: False
    - wf_time_pref: absolute
    - NI_ExpStartTimeStamp: 2017-06-20T11:44:43.000000
    - NI_ExpTimeStamp: 2017-06-20T11:44:43.000000
    - NI_ExpXDimension: t
    - wf_xname: Time
    - wf_xunit_string: s
    - NI_UnitDescription: uV
    - unit_string: uV
    - PhysicalMaximum: 90298.0
    - PhysicalMinimum: -90298.0
    - PreFilter: 
    - TransducerType: 
    - ValsPerRec: 128


In [96]:
with TdmsFile.read(r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 10\recording 1\Patient 10_1__final.tdms") as t:
    for g in t.groups():
        print("[Group]", g.name)
        for c in g.channels():
            print("  [Channel]", c.name, "samples:", len(c[:]))
            for k in c.properties:
                print(f"    - {k}: {c.properties[k]}")


[Group] Untitled
  [Channel] EKG samples: 132710144
    - wf_start_time: 2017-06-20T11:44:43.000000
    - wf_start_offset: 0.0
    - wf_increment: 0.001953125
    - DigitalMaximum: 32767
    - DigitalMinimum: -32768
    - NI_ChannelName: EKG
    - NI_ExpIsRelativeTime: False
    - wf_time_pref: absolute
    - NI_ExpStartTimeStamp: 2017-06-20T11:44:43.000000
    - NI_ExpTimeStamp: 2017-06-20T11:44:43.000000
    - NI_ExpXDimension: t
    - wf_xname: Time
    - wf_xunit_string: s
    - NI_UnitDescription: uV
    - unit_string: uV
    - PhysicalMaximum: 90298.0
    - PhysicalMinimum: -90298.0
    - PreFilter: 
    - TransducerType: 
    - ValsPerRec: 128


In [17]:
# from nptdms import TdmsFile, TdmsWriter, TdmsChannel
# from pathlib import Path
# from datetime import datetime
# import numpy as np
# import re

# def parse_time_from_string(s: str):
#     """
#     Forsøger fleksibelt at finde et timestamp i en tekst.
#     Understøtter fx:
#       - 2021-09-30 14:23:05
#       - 2021-09-30T14:23:05
#       - 30-09-2021 14:23:05
#       - 20210930_142305
#       - ...og varianter med -, _, / og mellemrum
#     Returnerer naive datetime (lokal tid) hvis fundet, ellers None.
#     """
#     candidates = []

#     # ISO-ish
#     m = re.search(r'(\d{4}-\d{2}-\d{2})[ T_](\d{2}:\d{2}:\d{2})', s)
#     if m:
#         candidates.append(f"{m.group(1)} {m.group(2)}")

#     # Kompakt YYYYMMDD_HHMMSS eller YYYYMMDDHHMMSS
#     m = re.search(r'(\d{8})[_-]?(\d{6})', s)
#     if m:
#         d, t = m.group(1), m.group(2)
#         candidates.append(f"{d[:4]}-{d[4:6]}-{d[6:]} {t[:2]}:{t[2:4]}:{t[4:]}")

#     # DMY fx 30-09-2021 14:23:05
#     m = re.search(r'(\d{2})[-_/](\d{2})[-_/](\d{4})[ T_](\d{2}):(\d{2}):(\d{2})', s)
#     if m:
#         day, mon, year, hh, mm, ss = m.groups()
#         candidates.append(f"{year}-{mon}-{day} {hh}:{mm}:{ss}")

#     # Ekstra: håndter formater som "30_09_2021 kl142305" eller "30_09_2021 kl_142305"
#     m = re.search(r'(\d{2})_(\d{2})_(\d{4})\s*kl_?(\d{2})(\d{2})(\d{2})', s)
#     if m:
#         day, mon, year, hh, mm, ss = m.groups()
#         candidates.append(f"{year}-{mon}-{day} {hh}:{mm}:{ss}")

#     # Prøv alle kandidater
#     for c in candidates:
#         try:
#             return datetime.strptime(c, "%Y-%m-%d %H:%M:%S")
#         except ValueError:
#             pass

#     # Sidste forsøg: ren dato uden tid -> sæt tid=00:00:00
#     m = re.search(r'(\d{4}-\d{2}-\d{2})', s)
#     if m:
#         try:
#             return datetime.strptime(m.group(1)+" 00:00:00", "%Y-%m-%d %H:%M:%S")
#         except ValueError:
#             pass

#     return None


In [18]:
# def inspect_tdms(tdms_path):
#     tdms_path = Path(tdms_path)
#     with TdmsFile.read(tdms_path) as tdms:
#         print(f"\n=== FILE: {tdms_path.name} ===")
#         print("File properties:")
#         for k, v in tdms.properties.items():
#             print(f"  - {k}: {v}")
#         for group in tdms.groups():
#             print(f"\n[Group] {group.name}")
#             # (Gruppe-properties kan ikke altid skrives 1:1 ud igen; vi bibeholder primært fil- og kanal-properties)
#             for ch in group.channels():
#                 print(f"  [Channel] {ch.name}")
#                 # Vis udvalgte properties
#                 prop_keys = list(ch.properties.keys())
#                 interesting = [p for p in prop_keys if p.lower() in {"wf_start_time", "wf_increment", "unit_string", "name"}]
#                 for p in interesting:
#                     print(f"    - {p}: {ch.properties[p]}")
#                 # Vis antal samples
#                 try:
#                     n = ch._length  # hurtigt, uden at læse alt
#                 except Exception:
#                     n = len(ch[:])   # fallback
#                 print(f"    - samples: {n}")


In [19]:
# def correct_tdms(
#     src_path,
#     dst_path,
#     corrected_by: str,
#     start_time_source: str = "file_property_name",  # "file_property_name" | "filename" | "exact_string"
#     start_time_exact_string: str = None,            # brug hvis start_time_source == "exact_string"
#     file_property_key: str = "name",                # brugt hvis start_time_source == "file_property_name"
#     preserve_original_start_time: bool = True
# ):
#     """
#     Læser src TDMS, sætter 'Corrected_by' på filniveau, opdaterer wf_start_time på alle kanaler der har det,
#     og skriver til dst TDMS. Skriver altid en liste af kanaler til TdmsWriter (ikke bare en gruppe),
#     så vi undgår TypeError i write_segment.
#     """
#     src_path = Path(src_path)
#     dst_path = Path(dst_path)
#     dst_path.parent.mkdir(parents=True, exist_ok=True)

#     with TdmsFile.read(src_path) as tdms:
#         # 1) Find korrekt starttid
#         if start_time_source == "file_property_name":
#             base = tdms.properties.get(file_property_key, "")
#             dt = parse_time_from_string(str(base))
#             if dt is None:
#                 raise ValueError(f"Kunne ikke parse tid fra file property '{file_property_key}': {base}")
#         elif start_time_source == "filename":
#             dt = parse_time_from_string(src_path.stem)
#             if dt is None:
#                 raise ValueError(f"Kunne ikke parse tid fra filnavn: {src_path.name}")
#         elif start_time_source == "exact_string":
#             if not start_time_exact_string:
#                 raise ValueError("Du valgte 'exact_string', men gav ingen streng i start_time_exact_string.")
#             dt = parse_time_from_string(start_time_exact_string)
#             if dt is None:
#                 raise ValueError(f"Kunne ikke parse tid fra given streng: {start_time_exact_string}")
#         else:
#             raise ValueError(f"Ukendt start_time_source: {start_time_source}")

#         # nptdms forventer numpy.datetime64 til wf_start_time
#         wf_start_np = np.datetime64(dt)

#         # 2) Forbered fil-properties
#         new_file_props = dict(tdms.properties) if tdms.properties else {}
#         new_file_props["Corrected_by"] = corrected_by
#         new_file_props["Corrected_on"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#         # 3) Byg nye kanaler (vi kopierer data + kanal-properties, og justerer wf_start_time hvor relevant)
#         channels_to_write = []
#         for group in tdms.groups():
#             for ch in group.channels():
#                 data = ch[:]  # læs hele kanalens data
#                 ch_props = dict(ch.properties) if ch.properties else {}
#                 if "wf_start_time" in ch_props:
#                     if preserve_original_start_time and "wf_start_time_original" not in ch_props:
#                         ch_props["wf_start_time_original"] = ch_props["wf_start_time"]
#                     ch_props["wf_start_time"] = wf_start_np
#                 # Konstruér en ny kanal med samme group/name + data + (opdaterede) props
#                 new_ch = ch.copy_with_new_properties(properties=ch_props)
#                 channels_to_write.append(new_ch)

#         # 4) Skriv ny TDMS — VIGTIGT: skriv KANALERNE, ikke kun en gruppe
#         with TdmsWriter(dst_path) as writer:
#             writer.write_segment(channels_to_write, file_properties=new_file_props)

#     return {
#         "dst": str(dst_path),
#         "parsed_start_time": str(dt),
#         "channels_written": len(channels_to_write)
#     }


In [20]:
# # Eksempel: tjek én fil inden du retter

# tdms_file = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 5\recording 1\Patient 5_1.tdms"
# inspect_tdms(tdms_file)


In [21]:
# src = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 5\recording 1\Patient 5_1.tdms"
# dst = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 5\recording 1\Patient 5_1_corrected.tdms"

# result = correct_tdms(
#     src_path=src,
#     dst_path=dst,
#     corrected_by="Kasper Vilken Jensen",
#     start_time_source="file_property_name",   # bruger file.properties["name"]
#     file_property_key="name"
# )
# print(result)


In [137]:
from nptdms import TdmsFile
from pathlib import Path
import re

src = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 3\recording 2\Henrik Koch 07092016_kl_105046__09092016_kl_103000.tdms"
dst_min = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 3\recording 2\Patient 3_2.tdms"

# >>> ANGIV KILDEFILEN HER <<<
SRC_PATH = Path(src)

def _parse_time_from_text(text: str):
    m = re.search(r'(\d{4}-\d{2}-\d{2})[ T_](\d{2}:\d{2}:\d{2})', text)
    if m: return f"{m.group(1)}T{m.group(2)}"
    m = re.search(r'(\d{8})[_-]?(\d{6})', text)  # fx 20170620_134443
    if m:
        d,t = m.groups()
        return f"{d[:4]}-{d[4:6]}-{d[6:]}T{t[:2]}:{t[2:4]}:{t[4:]}"
    return None

with TdmsFile.read(SRC_PATH) as tf:
    print(f"=== {SRC_PATH.name} ===")
    print("File properties:")
    for k,v in tf.properties.items():
        print(f"  - {k}: {v}")
    # evt. forslag til starttid ud fra file.properties["name"]
    suggested = _parse_time_from_text(str(tf.properties.get("name","")))
    if suggested:
        print(f"\n[Hint] Parsed starttid fra 'name': {suggested}")

    for g in tf.groups():
        print(f"\n[Group] {g.name}")
        for ch in g.channels():
            print(f"  [Channel] {ch.name}")
            # vis centrale props
            for k in ch.properties:
                print(f"    - {k}: {ch.properties[k]}")
            # antal samples (hurtigt estimat)
            try:
                n = ch._length
            except Exception:
                n = len(ch[:])
            print(f"    - samples: {n}")

# Initier en 'skrivningsplan' du kan redigere i næste celle
WRITE_PLAN = {
    "corrected_by": None,              # fx "Kasper Viken Jensen"
    "new_start_time": None,            # fx "2017-06-20 13:44:43" (bruges hvis sat)
    "use_name_property": True,         # brug parsed tid fra file.properties["name"] hvis new_start_time ikke er sat
    "file_prop_overrides": {},         # ekstra fil-properties du vil tilføje/rette
    "channel_prop_overrides": {},      # per-kanal overrides: {"GroupName/ChannelName": {"key": value, ...}, ...}
}
print("\n[OK] WRITE_PLAN initialiseret (redigér i næste celle).")


=== Henrik Koch 07092016_kl_105046__09092016_kl_103000.tdms ===
File properties:
  - name: Henrik Koch 07092016_kl_105046__09092016_kl_103000.tdms
  - author: Jesper
  - description: 
  - registertxt1: Written by National Instruments LabVIEW

[Group] Untitled
  [Channel] EKG
    - wf_start_time: 2016-09-07T08:50:46.000000
    - wf_start_offset: 0.0
    - wf_increment: 0.001953125
    - wf_samples: 86426624
    - DigitalMaximum: 32767
    - DigitalMinimum: -32768
    - NI_ChannelName: EKG
    - NI_ExpIsRelativeTime: False
    - wf_time_pref: absolute
    - NI_ExpStartTimeStamp: 2016-09-07T08:50:46.000000
    - NI_ExpTimeStamp: 2016-09-07T08:50:46.000000
    - NI_ExpXDimension: t
    - wf_xname: Time
    - wf_xunit_string: s
    - NI_UnitDescription: uV
    - unit_string: uV
    - PhysicalMaximum: 90298.0
    - PhysicalMinimum: -90298.0
    - PreFilter: 
    - TransducerType: 
    - ValsPerRec: 128
    - samples: 86426624

[OK] WRITE_PLAN initialiseret (redigér i næste celle).


In [138]:
from numpy import datetime64


corrected_time = datetime64('2016-09-07T10:50:46.000000')
print("Corrected time:", corrected_time)

Corrected time: 2016-09-07T10:50:46.000000


In [139]:
# Eksempler — RET som du vil:

# Hvem har rettet
WRITE_PLAN["corrected_by"] = "Kasper Viken Jensen"

# Sæt ny starttid eksplicit (foretrækkes for at undgå tz/DST-skred)
# (Hvis du lader den være None, og use_name_property=True, bruges parsed værdi fra file.properties["name"])
WRITE_PLAN["wf_start_time"] = corrected_time  # eller None
WRITE_PLAN["NI_ExpStartTimeStamp"] = corrected_time  # eller None
WRITE_PLAN["NI_ExpTimeStamp"] = corrected_time  # eller None

# Evt. ekstra fil-properties
WRITE_PLAN["file_prop_overrides"].update({
    "description": "Corrected start time & metadata",
    "name": "Patient 3_2 07_09_2016 kl_10_50_46__09_09_2016 kl_10_30_00",
})

# Evt. kanal-specifikke overrides (eksempel)
# key = "Group/Channel", fx "Untitled/EKG"
WRITE_PLAN["channel_prop_overrides"]["Untitled/EKG"] = {
    # "unit_string": "uV",
    # "wf_xunit_string": "s",
}

print("[OK] WRITE_PLAN opdateret:")
for k,v in WRITE_PLAN.items():
    print(" -", k, ":", v)


[OK] WRITE_PLAN opdateret:
 - corrected_by : Kasper Viken Jensen
 - new_start_time : None
 - use_name_property : True
 - file_prop_overrides : {'description': 'Corrected start time & metadata', 'name': 'Patient 3_2 07_09_2016 kl_10_50_46__09_09_2016 kl_10_30_00'}
 - channel_prop_overrides : {'Untitled/EKG': {}}
 - wf_start_time : 2016-09-07T10:50:46.000000
 - NI_ExpStartTimeStamp : 2016-09-07T10:50:46.000000
 - NI_ExpTimeStamp : 2016-09-07T10:50:46.000000


In [140]:
# === CELLE 3 — Skriv ny TDMS med rettelser fra WRITE_PLAN ===
from nptdms import TdmsFile, TdmsWriter
from nptdms.writer import RootObject, GroupObject, ChannelObject
from pathlib import Path
import numpy as np

# >>> ANGIV DESTINATIONSFIL HER <<<
DST_PATH = Path(dst_min)
DST_PATH.parent.mkdir(parents=True, exist_ok=True)

def _to_npdt64(ts):
    """Accepterer str / numpy.datetime64 / pandas.Timestamp / datetime -> np.datetime64 (naiv)."""
    if ts is None:
        return None
    if isinstance(ts, np.datetime64):
        return ts
    # str: tillad både "YYYY-MM-DD HH:MM:SS" og "YYYY-MM-DDTHH:MM:SS"
    if isinstance(ts, str):
        return np.datetime64(ts.replace(" ", "T"))
    # generisk fallback
    try:
        return np.datetime64(str(ts).replace(" ", "T"))
    except Exception:
        return None

with TdmsFile.read(SRC_PATH) as tf, TdmsWriter(DST_PATH) as writer:
    # Fil-properties
    file_props = dict(tf.properties) if tf.properties else {}
    if WRITE_PLAN.get("corrected_by"):
        file_props["Corrected_by"] = WRITE_PLAN["corrected_by"]
    file_props.update(WRITE_PLAN.get("file_prop_overrides", {}))

    # Forbered tidsfelter fra WRITE_PLAN (kan være None)
    plan_wf_start   = _to_npdt64(WRITE_PLAN.get("wf_start_time"))
    plan_exp_start  = _to_npdt64(WRITE_PLAN.get("NI_ExpStartTimeStamp"))
    plan_exp_time   = _to_npdt64(WRITE_PLAN.get("NI_ExpTimeStamp"))

    objs = [RootObject(file_props)]

    for g in tf.groups():
        g_props = dict(getattr(g, "properties", {}) or {})
        objs.append(GroupObject(g.name, properties=g_props))

        for ch in g.channels():
            data = np.array(ch[:], copy=True)            # detach fra memmap
            props = dict(ch.properties) or {}
            props.pop("wf_samples", None)                # afledes af datalængde

            # Bevar originaler hvis vi kommer til at overskrive
            if plan_wf_start is not None:
                if "wf_start_time_original" not in props:
                    props["wf_start_time_original"] = props.get("wf_start_time", None)
                props["wf_start_time"] = plan_wf_start

            if plan_exp_start is not None:
                if "NI_ExpStartTimeStamp_original" not in props:
                    props["NI_ExpStartTimeStamp_original"] = props.get("NI_ExpStartTimeStamp", None)
                props["NI_ExpStartTimeStamp"] = plan_exp_start

            if plan_exp_time is not None:
                if "NI_ExpTimeStamp_original" not in props:
                    props["NI_ExpTimeStamp_original"] = props.get("NI_ExpTimeStamp", None)
                props["NI_ExpTimeStamp"] = plan_exp_time

            # Cast evt. resterende NI-tidsfelter, hvis de er strenge
            for k in ("wf_start_time", "NI_ExpStartTimeStamp", "NI_ExpTimeStamp"):
                if k in props and isinstance(props[k], str):
                    try:
                        props[k] = np.datetime64(props[k].replace(" ", "T"))
                    except Exception:
                        pass

            # Kanal-specifikke overrides fra WRITE_PLAN (fx "Untitled/EKG")
            key = f"{g.name}/{ch.name}"
            for kk, vv in WRITE_PLAN.get("channel_prop_overrides", {}).get(key, {}).items():
                props[kk] = vv

            objs.append(ChannelObject(g.name, ch.name, data, properties=props))

    writer.write_segment(objs)

print(f"[OK] Skrev ny fil → {DST_PATH}")


[OK] Skrev ny fil → D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 3\recording 2\Patient 3_2.tdms


In [141]:
from nptdms import TdmsFile

def quick_check(path):
    from pathlib import Path
    with TdmsFile.read(path) as t:
        print(f"=== {Path(path).name} ===")
        print("File properties:", dict(t.properties))
        for g in t.groups():
            print("[Group]", g.name)
            for c in g.channels():
                print("  [Channel]", c.name, "samples:", len(c[:]))
                for k in c.properties:
                    print(f"    - {k}: {c.properties[k]}")

quick_check(DST_PATH)


=== Patient 3_2.tdms ===
File properties: {'name': 'Patient 3_2 07_09_2016 kl_10_50_46__09_09_2016 kl_10_30_00', 'author': 'Jesper', 'description': 'Corrected start time & metadata', 'registertxt1': 'Written by National Instruments LabVIEW', 'Corrected_by': 'Kasper Viken Jensen'}
[Group] Untitled
  [Channel] EKG samples: 86426624
    - wf_start_time: 2016-09-07T10:50:46.000000
    - wf_start_offset: 0.0
    - wf_increment: 0.001953125
    - DigitalMaximum: 32767
    - DigitalMinimum: -32768
    - NI_ChannelName: EKG
    - NI_ExpIsRelativeTime: False
    - wf_time_pref: absolute
    - NI_ExpStartTimeStamp: 2016-09-07T10:50:46.000000
    - NI_ExpTimeStamp: 2016-09-07T10:50:46.000000
    - NI_ExpXDimension: t
    - wf_xname: Time
    - wf_xunit_string: s
    - NI_UnitDescription: uV
    - unit_string: uV
    - PhysicalMaximum: 90298.0
    - PhysicalMinimum: -90298.0
    - PreFilter: 
    - TransducerType: 
    - ValsPerRec: 128
    - wf_start_time_original: 2016-09-07T08:50:46.000000
    